导包，定义常量

In [1]:
import json
import os

import requests

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
}

随便选择一首歌，就id为`22709528`的好了。得到它的元数据。

> 对于网易云音乐的id们，一般称歌曲id为`music_id`（简写为`mid`），专辑id为`album_id`

In [2]:

mid = 28921695
details = requests.get(
    "https://music.163.com/api/song/detail?ids=[%d]"%mid, # 可以在中括号中添加更多的音乐id，以逗号分隔
    headers=HEADERS
).json()
print(json.dumps(details, ensure_ascii=False, indent=4))

{
    "songs": [
        {
            "name": "Nine Point Eight",
            "id": 28921695,
            "position": 3,
            "alias": [],
            "status": 0,
            "fee": 8,
            "copyrightId": 663018,
            "disc": "1",
            "no": 7,
            "artists": [
                {
                    "name": "Mili",
                    "id": 339594,
                    "picId": 0,
                    "img1v1Id": 0,
                    "briefDesc": "",
                    "picUrl": "https://p1.music.126.net/6y-UleORITEDbvrOLV0Q8A==/5639395138885805.jpg",
                    "img1v1Url": "https://p1.music.126.net/6y-UleORITEDbvrOLV0Q8A==/5639395138885805.jpg",
                    "albumSize": 0,
                    "alias": [],
                    "trans": "",
                    "musicSize": 0,
                    "topicPerson": 0
                }
            ],
            "album": {
                "name": "H△G×Mili",
                "id": 2943236,

再获得它的歌词。

In [3]:
lyrics = requests.get(
    "https://music.163.com/api/song/lyric?id=%d&lv=-1&kv=-1&tv=-1" % mid,
    headers=HEADERS,
).json()
print(json.dumps(lyrics, ensure_ascii=False, indent=4))

{
    "sgc": false,
    "sfy": false,
    "qfy": false,
    "transUser": {
        "id": 3115294,
        "status": 99,
        "demand": 1,
        "userid": 70740635,
        "nickname": "都给我喝豆乳玉麒麟",
        "uptime": 1434682801393
    },
    "lyricUser": {
        "id": 3115293,
        "status": 99,
        "demand": 0,
        "userid": 55085075,
        "nickname": "十年一剑max",
        "uptime": 1434682801393
    },
    "lrc": {
        "version": 11,
        "lyric": "[00:12.500]You Ready?\n[00:24.780]Calla lily, carnation, daisy\n[00:27.060]Silently chase away your worries\n[00:30.600]Chrysanthemum, kalanchoe\n[00:32.920]Become your shield whenever you fall asleep\n[00:36.100]I cried out\n[00:38.530]Please don't leave me behind, leave me behind\n[00:41.510]So you held me tight\n[00:44.290]And said I will be just fine, I will be just fine, I will be just fine\n[00:50.960]Petals dance for our valediction\n[00:53.540]And synchronize to your frozen pulsation\n[00:56.980]Take me to wh

使用音乐元数据生成文件名

In [4]:
music = details["songs"][0]
artists = ",".join([a["name"] for a in music["artists"]])
title = music["name"]
filename = f"{artists} - {title}.lrc"
print(filename)

Mili - Nine Point Eight.lrc


对生成的文件名做处理，使它能当作一个文件名。

（这里采用将某些字符转换成对应的全角的方法）

> 就示例而言没什么要处理的，但是正常使用总会有的 ~~，比如就有这么一个叫`Leo/need`的名字里面就带了一个`/`，一定是来找茬的😡👊~~

In [5]:
filename = filename.translate(
    str.maketrans(
        {
            "/": "／",
            "*": "＊",
            ":": "：",
            "\\": "＼",
            ">": "＞",
            "<": "＜",
            "|": "｜",
            "?": "？",
            '"': "＂",
        }
    )
)
print(filename)

Mili - Nine Point Eight.lrc


写入文件

In [6]:
output_folder = "./"
with open(os.path.join(output_folder, filename), "w+", encoding="utf-8") as fp:
    fp.write(lyrics["lrc"]["lyric"])

似乎有些太简单了？

接下来我们利用网易云音乐的音乐搜索API，为本地文件匹配并下载歌词。在`samples`文件夹下有几个测试用的音频文件。

In [7]:
os.listdir("./samples/")

['01. せせらぎの向こうに見えた幻想.flac',
 'GET ON WiTH FUNKOT.mp3',
 'step on your heart.mp3',
 'やかもち,琴葉姉妹 - サカサカバンバンバスピスピス.mp3',
 '水野あつ,可不 - 無理に笑わなくて良いよ.mp3']

显然文件名的格式有很多种，不是很方便读入。虽然也可以分情况尽可能地讨论就是了。

但是除了文件名，我们还可以使用音乐标签。（没有标签的就只能尝试解析文件名了）

为了读取音乐标签，我们使用第三方库[`mutagen`](https://pypi.org/project/mutagen/)。安装然后导入它。

> 也有别的包可以用，比如[`tinytag`](https://github.com/tinytag/tinytag)，是一个很小巧的tag查询库，值得一试。
>
> ↑ 记得用最新的源代码而不是用release因为有bug（←至少到2024.07.27的1.10.1release里都还有），如果还是有的话记得去开issue

In [8]:
import mutagen

改变输入文件，查看音乐标签的不同情况。

In [9]:
folder = "./samples/"
file = mutagen.File(  #                     v 改变这里的index就行
    os.path.join(folder, os.listdir(folder)[2])
)
print(file.pprint())
if t := file.get("TIT2"):
    title = str(t)
else:
    title = None
if a := file.get("TPE1"):
    artists = list(a)
else:
    artists = []
print("[", title, "], by:", artists)

MPEG 1 layer 3, 128000 bps (CBR), 44100 Hz, 2 chn, 137.98 seconds (audio/mp3)
COMM==XXX=163 key(Don't modify):L64FU3W4YxX3ZFTmbZ+8/d9/yL6bJXf+RTFn1xL8d2ti0KfLGUG3Nl2WPvz0BCk+S7z/PbupBQwQC9ETcxkE2RXlMREF7cfRQ5QjEhNKbzrnHxwUbBe4B/H1RuxCs1MKE/wQ17Y1wSXdvsAN2KytOFsqgCG86IvBXwJ0l+7vg152QM6fTfPWVfFesJEBwGxs+dUleNsCJMOw7qV2zP8/db+CH5/6owzSQDXIVloNTQNbMJfM7RW6M2uqnMUz/5xoxY28kxj7qGvQaD+QX3ABvb2t40+RHolmidiyj90Kr+ZuOkAKTf+km5Nt7bOa/bOinuGTbYdHoQUyhgcMqqGllmugvIqNZ0HAFOgJGJF3qCKrL6/OtBsixsqFU4cL+NbkFcx8U+kuI7TWAsbFxgst46JwPpezaBkQz3dL0u4HedR/oJz6GvazBehAAmRXWsrEsaY5uHSrK6Hp1JI7a5slIMt3CYgmsaGXs0lsB0+/j7bHUtM4Q9k+tby0BcJgbTtR6YURAByiRHQ6CI5tvtxzQg==
COMM=ID3v1 Comment=eng=163 key(Don't modify):L64FU3
TIT2=step on your heart
TPE1=洛天依
TPOS=1
TRCK=1
TSSE=Lavf56.4.101
[ step on your heart ], by: ['洛天依']


对于MP3文件，可以从`TIT2`和`TPE1`字段分别拿到标题和艺术家。

关于各种格式的字段名可参见[EXIF Tools](https://exiftool.org/TagNames/)，不同格式音频文件之间的字段名可能不一致，比如上面那段代码就没法处理FLAC文件的标签 ~~（这个时候tinytag的方便之处就体现出来了，赞美喵）~~

此外还需要注意返回值为`None`的问题。还有部分文件的艺术家格式可能不规范，需要取第一项用`/`符之类的东西再切一刀。

不论是从文件名还是从文件标签，总之我们有多种方法拿到音乐文件的标题与艺术家（可以做一个fallback机制？）。我们就拿这些字段去请求搜索接口，然后匹配一致（模糊匹配当然也可以）的搜索结果，然后从对应的`music_id`下载歌词就好了。

> 注意到大部分从网易云音乐下载的音乐文件的注释（`COMM`或`DESCRIPTION`字段）都有一大串以`163 key(Don't modify):`开头的字符。这是网易云用来存储音乐元数据的方式，内容是以base64形式存储的以AES-128-ECB算法加密的json字符串，密钥为`#14ljk_!\]&0U<'(`，解密后可以直接得到`mid`。网上也有很多资料，试着自己去探索吧！

假设我们现在得到了这么一组信息，接下来就拿这组信息去请求搜索接口了。

In [10]:
title = "無理に笑わなくて良いよ"
artists = ["水野あつ", "可不"]

In [11]:
from urllib import parse

result = requests.get("https://music.163.com/api/search/get/", params={
    "s": "+".join([title] + artists),
    "limit": 10,
    "type": 1,
    "offset": 0
}, headers=HEADERS).json()
print(json.dumps(result, ensure_ascii=False, indent=4))

{
    "result": {
        "songs": [
            {
                "id": 1922287635,
                "name": "無理に笑わなくて良いよ",
                "artists": [
                    {
                        "id": 37446417,
                        "name": "水野あつ",
                        "picUrl": null,
                        "alias": [],
                        "albumSize": 0,
                        "picId": 0,
                        "fansGroup": null,
                        "img1v1Url": "https://p1.music.126.net/6y-UleORITEDbvrOLV0Q8A==/5639395138885805.jpg",
                        "img1v1": 0,
                        "trans": null
                    },
                    {
                        "id": 47893107,
                        "name": "可不",
                        "picUrl": null,
                        "alias": [],
                        "albumSize": 0,
                        "picId": 0,
                        "fansGroup": null,
                        "img1v1Url": "https:

In [12]:
music = result["result"]["songs"][0]
mid = music["id"]

如果匹配成功，就拿着得到的音乐id去获取歌词就好了。这里就略过了。

这里有一个简单的163key解密示例，需要`pycryptodomex`库。

```powershell
pip install pycryptodomex
```

In [13]:
from Cryptodome.Cipher import AES
import base64


def decrypt(data: bytes) -> bytes:
    key = "#14ljk_!\\]&0U<'(".encode("utf-8")
    cryptor = AES.new(key, AES.MODE_ECB)
    return cryptor.decrypt(data)


sample_key = "L64FU3W4YxX3ZFTmbZ+8/fMZ5HO+EQE0d1hry/01n1imgJAjwEU1sIK3DunPCv8JIDfFiwO6gpGCI/omCPk6o+iyDgemCERB74r1dmvVr8Wbogz195VQ2TusMWpQYrr4hv/5sNK3RLF63c1W6YvLbyvK6F1ZqljV++8nuxUKMajm+9C5eM6uLnnXsqxoZ7z6YaVNWdj+A3FGInsVRPMIxls7Lz9KplmWAGoRrSj/P2lheZZvLdikmlDL64LPYugLkxcTQfHFwdDa3NDBPsBwgNldMjhmdKGHER7CrMKF16c+IIias7TFLviRxlj1RDghodu6aXmBkJFBWP8l+2zsTlxS+EaX64C2/+fAQ7mfvO4FZvxwKtRKs+o5oQ2tsNJSvdTNTzUv8ByWjQAEvlmpzRUFACo2gQuRRis/3UGGpp78C/oUnwwIsHvW3oczICzgQBqC8V3o4OHTuKyaqOM3rb6j3NYr7FgyLwLqKc5lJlW8XfeKKdsSJkQCD4MLZqMC/hmTRvL/Ob7ebNgLIGdmL5ClgoxqZ2mphA5EFWWFZydpvvxRrsv8frGMDb/Hd/EdB11TFPcrhCcM9Vd5l+xznQ=="
dec = decrypt(base64.b64decode(sample_key.encode(encoding="utf-8"))).decode(
    encoding="utf-8"
)[6:]
# print(dec)
meta = json.loads(dec[: dec.rfind("}") + 1])
print(json.dumps(meta, ensure_ascii=False, indent=4))

{
    "musicId": 785882,
    "musicName": "せせらぎの向こうに見えた幻想",
    "artist": [
        [
            "みかん箱",
            20578
        ],
        [
            "Foxtail-Grass Studio",
            19481
        ]
    ],
    "albumId": 77670,
    "album": "pastoral landscape",
    "albumPicDocId": 4455221115751031,
    "albumPic": "https://p3.music.126.net/JFnBz-eyIBltKt15wjjFUA==/4455221115751031.jpg",
    "bitrate": 1029000,
    "mp3DocId": "3988c6adcd44acba4094a9a7e0ae987f",
    "duration": 176386,
    "mvId": 0,
    "alias": [],
    "transNames": [],
    "format": "flac"
}


此外我们有时还会需要下载一整个专辑的音乐的歌词。我目前没有找到未加密的接口用于获取专辑列表。但是专辑列表在网页的源代码中能找到，而且还是以JSON形式存在于一个textarea中，id为`song-list-pre-data`，好怪噢。

虽然一下子就能写出它的XPath然后`lxml`启动，但是我就是要启动`BeautifulSoup` ~~，豪丸！~~

> `pip install beautifulsoup4`

假设我们要下载`2084285`这个专辑的所有歌词。先得到专辑中所有的音乐的id。

In [14]:
from bs4 import BeautifulSoup as BS

aid = 2084285
bs = BS(
    requests.get(f"https://music.163.com/album?id={aid}", headers=HEADERS).text,
    "html.parser",
)
album = json.loads(bs.find("textarea", id="song-list-pre-data").get_text())
print(json.dumps(album, ensure_ascii=False, indent=4))

[
    {
        "privilege": {
            "id": 22709523,
            "fee": 0,
            "payed": 0,
            "st": 0,
            "pl": 320000,
            "dl": 999000,
            "sp": 7,
            "cp": 1,
            "subp": 1,
            "cs": false,
            "maxbr": 999000,
            "fl": 320000,
            "toast": false,
            "flag": 256,
            "preSell": false,
            "playMaxbr": 999000,
            "downloadMaxbr": 999000,
            "maxBrLevel": "lossless",
            "playMaxBrLevel": "lossless",
            "downloadMaxBrLevel": "lossless",
            "plLevel": "exhigh",
            "dlLevel": "lossless",
            "flLevel": "exhigh",
            "rscl": null,
            "freeTrialPrivilege": {
                "resConsumable": false,
                "userConsumable": false,
                "listenType": null,
                "cannotListenReason": null,
                "playReason": null,
                "freeLimitTagType": nu

In [15]:
mids = [int(i["id"]) for i in album]
print(mids)

[22709523, 22709533, 22709527, 22709526, 22709524, 22709535, 22709530, 22709529, 22709525, 22709536, 22709532, 22709531, 22709528, 22709537, 22709534]


很好我们现在得到了所有的音乐id。可以将它们全部下载下来了。

> 但是你有没有注意到，在得到的专辑列表以及先前得到的搜索结果中，除了音乐id，还包含了每首音乐的标题与艺术家。也就是说没有必要再去请求一遍每首音乐的信息了 —— 好欸！

现在让我们回到README中。